In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Function which calculates number of win and lose trades in a row

def win_lose_streak(df: pd.DataFrame)-> float:
    win = 1
    new_win = 1
    lose = -1
    new_lose = -1
    win_list =[]
    lose_list = []
    total_list = []
    for trade in range(1, len(df["profit"])):
        if df["profit"].iloc[0] > 0:
            win = 1
            win_list.append(win)
        if df["profit"].iloc[0] < 0:
            lose = -1
            lose_list.append(lose)
        if df["profit"].iloc[trade] > 0 and df["profit"].iloc[trade - 1] > 0:
            win += 1
            win_list.append(win)
            total_list.append(new_win)
            if new_win < win:
                new_win = win
        if df["profit"].iloc[trade] < 0 and df["profit"].iloc[trade - 1] < 0:
            lose -= 1
            lose_list.append(lose)
            total_list.append(new_lose)
            if new_lose < lose:
                new_lose = lose
        if df["profit"].iloc[trade] > 0 and df["profit"].iloc[trade - 1] < 0:
            win = 1
            win_list.append(win)
        if df["profit"].iloc[trade] < 0 and df["profit"].iloc[trade - 1] > 0:
            lose = -1
            lose_list.append(lose)
        
    average_win_streak = {'average_win_streak' : sum(win_list) / len(win_list)}
    average_lose_streak = {'average_lose_streak': sum(lose_list) / len(lose_list)}
    max_win =  {'max_wins': max(win_list)}
    max_lose = {'max_lose': min(lose_list)}
    return [max_win, max_lose, average_win_streak, average_lose_streak]

In [14]:
class Load_data:
    """
    Class loads data, preprocess it, make calculations of strategy performance and displays statistics.
    """
    def __init__(self, data):
        """
        Method load data from csv to dataframe
        """
        self.df = pd.read_csv(data)
        
    def change_time_format(self):
        """
        Method change format of time columns
        """
        self.df["time"] = pd.to_datetime(self.df["time"].apply(lambda x: x.split('+')[0].replace('T', ' ')))
        return self
        
    def drop_rename_cols(self):
        """
        Method drops useless columns and rename required columns
        """
        self.df.dropna(axis = 1, how="all", inplace = True)
        self.df.drop(labels =["Plot.3", "Plot.4"], axis = 1, inplace = True)
        self.df.rename({"Plot":"Upper band", "Plot.1":"Lower band"}, axis = "columns", inplace = True)
        return self
        
    def long_buys(self):
        """
        Method reformats dataframe to use datapoints at the time of entry into the trade
        """
        self.df = self.df[self.df["Long/Buy"].notna()]
        return self
    
    def calculate_distance_to_sl(self):
        """
        Method caluclate distance to stoploss at the time of entry into the trade
        """
        self.df["Avg_%_to_SL"] = np.nan
        self.df.iloc[:, 1:]
        self.df["Avg_%_to_SL"] = (self.df["Lower band"] - self.df["close"]) / self.df["close"]
        return self
    
    def calculate_profit(self):
        """
        Method creates dataframe with data from entry and exit trades to calculate profit
        """
        self.df = self.df[(self.df["Long/Buy"].notna()) | (self.df["Short/Sell"].notna())]
        self.df["sell breakout"] = self.df["sell breakout"].shift(-1)
        self.df = self.df[(self.df["Buy breakout"].notna())]
        self.df["profit"] = np.nan
        self.df["profit"] = (self.df["sell breakout"] - self.df["Buy breakout"]) / self.df["Buy breakout"] * 100
        self.df["gross_profit"] = np.nan
        self.df["gross_loss"] = np.nan
        self.df["gross_profit"] = self.df["profit"].apply(lambda x: x if x > 0 else 0)
        self.df["gross_loss"] = self.df["profit"].apply(lambda x: x if x < 0 else 0)
        return self
        
        
    def display_statistics(self, param_to_show = ["Distance_to_SL", "Profit"]):
        """
        Metohd displays statistics for both Distance to stop-loss and Profit variations. 
        The statistics is displayed for the whole available period and more detailed statistics 
        of predefined periods corresponding to market cycles. 
        """
        
        temp_2013_2015 = self.df.loc[(self.df["time"] > "2013-12-05") & (self.df["time"] < "2015-11-01")]
        temp_2015_2017 = self.df.loc[(self.df["time"] > "2015-11-01") & (self.df["time"] < "2017-12-16")]
        temp_2018_2019 = self.df.loc[(self.df["time"] > "2017-12-17") & (self.df["time"] < "2019-01-22")]
        temp_2019_2021 = self.df.loc[(self.df["time"] > "2019-01-23") & (self.df["time"] < "2021-04-12")]
        temp_2021_2022 = self.df.loc[(self.df["time"] > "2021-04-13")]
        periods = [temp_2013_2015, temp_2015_2017, temp_2018_2019, temp_2019_2021, temp_2021_2022]
        
        if param_to_show == "Distance_to_SL":
            print(f"Total stop-loss statistics from  {self.df['time'].iloc[0]} to {self.df['time'].iloc[-1]}\n\n"
                  f"Mean stop-loss %: {round(self.df['Avg_%_to_SL'].describe()[1] * 100, 2)}\n"
                  f"Standard deviation %: {round(self.df['Avg_%_to_SL'].describe()[2] *100, 2)}\n"
                  f"Maximum stop-loss %: {round(self.df['Avg_%_to_SL'].describe()[3] * 100, 2)}\n"
                  f"Minimum stop-loss %: {round(self.df['Avg_%_to_SL'].describe()[7] * 100, 2)}\n")
        
        
            for frame in periods:
                if len(frame) == 0:
                    pass
                elif frame["time"].iloc[0] == self.df["time"].iloc[0]:
                    pass
                else:
                    print(f"Total stop-loss statistics from  {frame['time'].iloc[0]} to {frame['time'].iloc[-1]}\n\n"
                          f"Mean stop-loss %: {round(frame['Avg_%_to_SL'].describe()[1] * 100, 2)}\n"
                          f"Standard deviation %: {round(frame['Avg_%_to_SL'].describe()[2] *100, 2)}\n"
                          f"Maximum stop-loss %: {round(frame['Avg_%_to_SL'].describe()[3] * 100, 2)}\n"
                          f"Minimum stop-loss %: {round(frame['Avg_%_to_SL'].describe()[7] * 100, 2)}\n")
        
        elif param_to_show == "Profit":
            streaks = win_lose_streak(self.df)
            print(f"Total profit statistics from  {self.df['time'].iloc[0]} to {self.df['time'].iloc[-1]}\n\n"
                  f"Mean average trade %: {round(self.df['profit'].mean(), 2)}\n"
                  f"Mean winnig trade trade %: {round(self.df['gross_profit'].mean(), 2)}\n"
                  f"Mean Losing trade trade %: {round(self.df['gross_loss'].mean(), 2)}\n"
                  f"Gross profit %: {round(self.df['gross_profit'].sum(), 2)}\n"
                  f"Gross loss %: {round(self.df['gross_loss'].sum(), 2)}\n"
                  f"Risk/reward %: {round((self.df['gross_loss'].sum() / self.df['gross_profit'].sum() * -100), 2)}\n\n"
                  f"Max win streak: {streaks[0]['max_wins']}\n"
                  f"Max lose streak: {streaks[1]['max_lose']}\n"
                  f"Total win trades: {self.df['gross_profit'].count()}\n"
                  f"Total lose trades: {self.df['gross_loss'].count()}\n")            
           
            for frame in periods:
                if len(frame) == 0:
                    pass
                elif frame["time"].iloc[0] == self.df["time"].iloc[0]:
                    pass
                else:
                    print(f"Total profit statistics from  {frame['time'].iloc[0]} to {frame['time'].iloc[-1]}\n\n"
                          f"Mean average trade %: {round(frame['profit'].mean(), 2)}\n"
                          f"Mean winnig trade trade %: {round(frame['gross_profit'].mean(), 2)}\n"
                          f"Mean Losing trade trade %: {round(frame['gross_loss'].mean(), 2)}\n"
                          f"Gross profit %: {round(frame['gross_profit'].sum(), 2)}\n"
                          f"Gross loss %: {round(frame['gross_loss'].sum(), 2)}\n"
                          f"Risk/reward %: {round((frame['gross_loss'].sum() / self.df['gross_profit'].sum() * -100), 2)}\n\n"
                          f"Max win streak: {streaks[0]['max_wins']}\n"
                          f"Max lose streak: {streaks[1]['max_lose']}\n"
                          f"Total win trades: {frame['gross_profit'].count()}\n"
                          f"Total lose trades: {frame['gross_loss'].count()}\n")
                    
        

In [15]:
## Profit caluclation and displaying statistics
data = Load_data(".\ETHBTC_4H.csv")
data = data.change_time_format()
data = data.drop_rename_cols()
data = data.calculate_profit()
data.display_statistics(param_to_show="Profit")

Total profit statistics from  2015-09-20 23:00:00 to 2022-10-26 07:00:00

Mean average trade %: 24.5
Mean winnig trade trade %: 29.99
Mean Losing trade trade %: -5.49
Gross profit %: 1379.62
Gross loss %: -252.47
Risk/reward %: 18.3

Max win streak: 3
Max lose streak: -2
Total win trades: 46
Total lose trades: 46

Total profit statistics from  2015-11-04 22:00:00 to 2017-12-12 14:00:00

Mean average trade %: 42.28
Mean winnig trade trade %: 48.76
Mean Losing trade trade %: -6.48
Gross profit %: 1072.78
Gross loss %: -142.65
Risk/reward %: 10.34

Max win streak: 3
Max lose streak: -2
Total win trades: 22
Total lose trades: 22

Total profit statistics from  2018-04-10 23:00:00 to 2018-12-21 10:00:00

Mean average trade %: 5.29
Mean winnig trade trade %: 10.55
Mean Losing trade trade %: -5.26
Gross profit %: 42.2
Gross loss %: -21.05
Risk/reward %: 1.53

Max win streak: 3
Max lose streak: -2
Total win trades: 4
Total lose trades: 4

Total profit statistics from  2019-02-10 22:00:00 to 202

In [16]:
## Stolloss caluclation and displaying statistics
data = Load_data(".\ETHBTC_4H.csv")
data = data.change_time_format()
data = data.drop_rename_cols()
data = data.long_buys()
data = data.calculate_distance_to_sl()
data.display_statistics(param_to_show="Distance_to_SL")

Total stop-loss statistics from  2015-09-20 23:00:00 to 2022-10-26 07:00:00

Mean stop-loss %: -11.52
Standard deviation %: 1.73
Maximum stop-loss %: -19.23
Minimum stop-loss %: -10.03

Total stop-loss statistics from  2015-11-04 22:00:00 to 2017-12-12 14:00:00

Mean stop-loss %: -12.12
Standard deviation %: 2.15
Maximum stop-loss %: -19.23
Minimum stop-loss %: -10.14

Total stop-loss statistics from  2018-04-10 23:00:00 to 2018-12-21 10:00:00

Mean stop-loss %: -11.22
Standard deviation %: 1.27
Maximum stop-loss %: -13.08
Minimum stop-loss %: -10.3

Total stop-loss statistics from  2019-02-10 22:00:00 to 2021-03-31 23:00:00

Mean stop-loss %: -10.91
Standard deviation %: 1.0
Maximum stop-loss %: -13.06
Minimum stop-loss %: -10.03

Total stop-loss statistics from  2021-06-30 19:00:00 to 2022-10-26 07:00:00

Mean stop-loss %: -10.83
Standard deviation %: 0.96
Maximum stop-loss %: -12.71
Minimum stop-loss %: -10.07

